In [4]:
%load_ext dotenv
%dotenv ../.env

In [5]:
import os
from klaviyo_api import KlaviyoAPI

In [6]:
klaviyo = KlaviyoAPI(os.environ["KLAVIYO_API_KEY"])

In [7]:
klaviyo.Metrics.get_metrics() 

{'data': [{'type': 'metric',
   'id': 'R6pkzd',
   'attributes': {'name': 'Bounced Email',
    'created': '2024-04-29T21:05:18+00:00',
    'updated': '2024-04-29T21:05:18+00:00',
    'integration': {'object': 'integration',
     'id': '0rG4eQ',
     'key': 'klaviyo',
     'name': 'Klaviyo',
     'category': 'Internal'}},
   'links': {'self': 'https://a.klaviyo.com/api/metrics/R6pkzd/'}},
  {'type': 'metric',
   'id': 'RHqpbQ',
   'attributes': {'name': 'Opened Email',
    'created': '2024-04-29T21:05:18+00:00',
    'updated': '2024-04-29T21:05:18+00:00',
    'integration': {'object': 'integration',
     'id': '0rG4eQ',
     'key': 'klaviyo',
     'name': 'Klaviyo',
     'category': 'Internal'}},
   'links': {'self': 'https://a.klaviyo.com/api/metrics/RHqpbQ/'}},
  {'type': 'metric',
   'id': 'S6xELh',
   'attributes': {'name': 'Received Automated Response SMS',
    'created': '2024-04-29T21:05:22+00:00',
    'updated': '2024-04-29T21:05:22+00:00',
    'integration': {'object': 'integra

In [17]:
klaviyo.Campaigns.get_campaigns("equals(messages.channel,'email')")

{'data': [{'type': 'campaign',
   'id': '01HWNTWW47VMBR99KEDW5N9YBR',
   'attributes': {'name': 'Internal Klaviyo - Test Campaign Name',
    'status': 'Sent',
    'archived': False,
    'audiences': {'included': ['WgYJhH'], 'excluded': []},
    'send_options': {'use_smart_sending': False, 'ignore_unsubscribes': False},
    'tracking_options': {'is_add_utm': False,
     'utm_params': [],
     'is_tracking_clicks': True,
     'is_tracking_opens': True},
    'send_strategy': {'method': 'immediate',
     'options_static': None,
     'options_throttled': None,
     'options_sto': None},
    'created_at': '2024-04-29T21:06:43.464192+00:00',
    'scheduled_at': None,
    'updated_at': '2024-04-29T21:06:43.464231+00:00',
    'send_time': None},
   'relationships': {'campaign-messages': {'data': [{'type': 'campaign-message',
       'id': '01HWNTWW4CNE8MH82QZDTD4EE9'}],
     'links': {'self': 'https://a.klaviyo.com/api/campaigns/01HWNTWW47VMBR99KEDW5N9YBR/relationships/campaign-messages/',
     

### Create a template using the newletter archive in the database

In [8]:
from prisma import Prisma
db = Prisma()
await db.connect()

In [9]:
emails = await db.email.find_many(order=
    {
        "createdAt": "desc",
    }, take=10)

In [10]:
e = emails[3]

In [11]:
e.name

'Subject: Unlock the Secret to Radiant Health and Delicious Meals!'

In [19]:
body = {
    "data": {
        "type": "template",
        "attributes": {
          "name": e.name,
          "editor_type": "CODE",
          "html": e.html.replace("\n", "<br/>"),
          "text": e.text
        }
    }
}

In [20]:
template = klaviyo.Templates.create_template(body)

In [21]:
template

{'data': {'type': 'template',
  'id': 'Y67A2A',
  'attributes': {'name': 'Subject: Unlock the Secret to Radiant Health and Delicious Meals!',
   'editor_type': 'CODE',
   'html': '<html><head></head><body><p>Subject: Unlock the Secret to Radiant Health and Delicious Meals!<br/><br/>Hey there, fellow foodie!<br/><br/>Are you tired of feeling sluggish, bloated, and stuck in a culinary rut? Do you dream of having radiant skin, a robust immune system, and a kitchen that\'s always buzzing with flavor and excitement?<br/><br/>Let\'s face it: we\'ve all been there. We\'ve all struggled with digestive issues, joint pain, and the frustration of cooking the same old recipes over and over again. But what if I told you there\'s a game-changer waiting in the wings?<br/><br/>I\'m talking about the incredible, the amazing, the utterly fantastic Fully Cooked Beef Gelatin from Grass-fed, Grass-finished Beef! This isn\'t just any old gelatin, my friend. Oh no! This is the crème de la crème (pun intended

In [22]:
template_id = template["data"]["id"]; template_id

'Y67A2A'

OK great, now you can see the template here, under the "Saved templates" tab https://www.klaviyo.com/templates/list

### Create a Campaign using the template

In [23]:
lists = klaviyo.Lists.get_lists(filter="equals(name,'Sample Data List')")

In [24]:
lists

{'data': [{'type': 'list',
   'id': 'WgYJhH',
   'attributes': {'name': 'Sample Data List',
    'created': '2024-04-29T21:06:41+00:00',
    'updated': '2024-04-29T21:06:41+00:00',
    'opt_in_process': 'double_opt_in'},
   'relationships': {'profiles': {'links': {'self': 'https://a.klaviyo.com/api/lists/WgYJhH/relationships/profiles/',
      'related': 'https://a.klaviyo.com/api/lists/WgYJhH/profiles/'}},
    'tags': {'links': {'self': 'https://a.klaviyo.com/api/lists/WgYJhH/relationships/tags/',
      'related': 'https://a.klaviyo.com/api/lists/WgYJhH/tags/'}}},
   'links': {'self': 'https://a.klaviyo.com/api/lists/WgYJhH/'}}],
 'links': {'self': 'https://a.klaviyo.com/api/lists/?filter=equals%28name%2C%27Sample%20Data%20List%27%29',
  'next': None,
  'prev': 'https://a.klaviyo.com/api/lists/?filter=equals%28name%2C%27Sample+Data+List%27%29&page%5Bcursor%5D=%3CFilters%3A+equals%28name%2C%27Sample+Data+List%27%29%3E'}}

In [25]:
list_id = lists["data"][0]["id"]; list_id

'WgYJhH'

In [36]:
preview_text = template["data"]["attributes"]["text"][:100]

In [32]:
name = template["data"]["attributes"]["name"]; name

'Subject: Unlock the Secret to Radiant Health and Delicious Meals!'

In [37]:
body = {
    "data": {
        "type": "campaign",
        "attributes": {
            "name": name,
            "audiences": {
                "included": [list_id],
                "send_strategy": "immediate",
            },
            "campaign-messages": {
                "data": [
                    {
                        "type": "campaign-message",
                        "attributes": {
                            "channel": "email",
                            "label": name,
                            "content": {
                                "subject": name,
                                "preview_text": preview_text,
                                "from_email": "store@my-company.com",
                                "from_label": "My Company",
                                "reply_to_email": "reply-to@my-company.com",
                                "cc_email": "cc@my-company.com",
                                "bcc_email": "bcc@my-company.com",
                            },
                            "render_options": {
                                "shorten_links": True,
                                "add_org_prefix": True,
                                "add_info_link": True,
                                "add_opt_out_language": False,
                            },
                        },
                    }
                ]
            },
        },
    }
}

In [38]:
campaign = klaviyo.Campaigns.create_campaign(body); campaign

{'data': {'type': 'campaign',
  'id': '01HXVRFVAY2J0AD31QW4QF7K98',
  'attributes': {'name': 'Subject: Unlock the Secret to Radiant Health and Delicious Meals!',
   'status': 'Draft',
   'archived': False,
   'audiences': {'included': ['WgYJhH'], 'excluded': []},
   'send_options': {'use_smart_sending': True, 'ignore_unsubscribes': False},
   'tracking_options': {'is_add_utm': False,
    'utm_params': [],
    'is_tracking_clicks': True,
    'is_tracking_opens': True},
   'send_strategy': {'method': 'immediate',
    'options_static': None,
    'options_throttled': None,
    'options_sto': None},
   'created_at': '2024-05-14T14:35:47.935504+00:00',
   'scheduled_at': None,
   'updated_at': '2024-05-14T14:35:48.012830+00:00',
   'send_time': None},
  'relationships': {'campaign-messages': {'data': [{'type': 'campaign-message',
      'id': '01HXVRFVB3378AX2XS98GHCPH6'}],
    'links': {'self': 'https://a.klaviyo.com/api/campaigns/01HXVRFVAY2J0AD31QW4QF7K98/relationships/campaign-messages/',

Associate the campaign with a template

In [41]:
campaign_id = campaign["data"]["relationships"]["campaign-messages"]["data"][0]["id"]; campaign_id

'01HXVRFVB3378AX2XS98GHCPH6'

In [42]:
body =  {
    "data": {
        "type": "campaign-message",
        "id": campaign_id,
        "relationships": {"template": {"data": {"type": "template", "id": template_id}}},
    }
}

In [43]:
klaviyo.Campaigns.create_campaign_message_assign_template(body)